In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import tensorflow

In [134]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\huyen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\huyen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\huyen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [151]:
df = pd.read_csv('mr/sst2.csv', encoding='unicode_escape', header=0)

In [152]:
df.head(5)

,sentiment,review,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,0,one long string of cliches .,NaN,NaN,NaN,NaN,NaN
1,0,if you 've ever entertained the notion of doin...,NaN,NaN,NaN,NaN,NaN
2,0,k-19 exploits our substantial collective fear ...,NaN,NaN,NaN,NaN,NaN
3,0,it 's played in the most straight-faced fashio...,NaN,NaN,NaN,NaN,NaN
4,1,"there is a fabric of complex ideas here , and ...",NaN,NaN,NaN,NaN,NaN


In [153]:
df.info() # Basic Information
print("\nShape: ",df.shape) # Shape of data
print("\nChecking for missing values")
df.isnull().sum()# Check for missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9613 entries, 0 to 9612
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sentiment   9613 non-null   int64 
 1   review      9613 non-null   object
 2   Unnamed: 2  333 non-null    object
 3   Unnamed: 3  137 non-null    object
 4   Unnamed: 4  37 non-null     object
 5   Unnamed: 5  5 non-null      object
 6   Unnamed: 6  2 non-null      object
dtypes: int64(1), object(6)
memory usage: 525.8+ KB

Shape:  (9613, 7)

Checking for missing values


sentiment        0
review           0
Unnamed: 2    9280
Unnamed: 3    9476
Unnamed: 4    9576
Unnamed: 5    9608
Unnamed: 6    9611
dtype: int64

In [154]:
df.head(5)

,sentiment,review,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,0,one long string of cliches .,NaN,NaN,NaN,NaN,NaN
1,0,if you 've ever entertained the notion of doin...,NaN,NaN,NaN,NaN,NaN
2,0,k-19 exploits our substantial collective fear ...,NaN,NaN,NaN,NaN,NaN
3,0,it 's played in the most straight-faced fashio...,NaN,NaN,NaN,NaN,NaN
4,1,"there is a fabric of complex ideas here , and ...",NaN,NaN,NaN,NaN,NaN


In [155]:
#Dropping unnecessary columns
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6'],axis=1,inplace=True)

In [156]:
df.to_csv('mr/sst2.csv', index=False)

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    print('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1080 Ti


In [3]:
import pandas as pd

In [4]:
from numpy.random import RandomState
#%% Split whole dataset into train and valid set
df = pd.read_csv('mr/sst2.csv', encoding='unicode_escape', header=0)
rng = RandomState()

train = df.sample(frac=0.9, random_state=rng)
test = df.loc[~df.index.isin(train.index)]
train.to_csv('mr/train-sst2.csv', index=False)
test.to_csv('mr/test-sst2.csv', index=False)

In [5]:
train_df = pd.read_csv("mr/train-sst2.csv", encoding='unicode_escape', header=0)

In [6]:
train_df

,sentiment,review
0,0,feels more like a rejected x-files episode tha...
1,0,with spy kids 2 : the island of lost dreams wr...
2,1,"while the story 's undeniably hard to follow ,..."
3,1,"not only is undercover brother as funny , if n..."
4,0,dignified ceo 's meet at a rustic retreat and ...
...,...,...
8647,1,a superbly acted and funny\/gritty fable of th...
8648,0,we miss the quirky amazement that used to come...
8649,0,a benign but forgettable sci-fi diversion .
8650,1,elvira fans could hardly ask for more .


In [7]:
train_df.iloc[0][1]

C:\Users\huyen\AppData\Local\Temp\ipykernel_12436\1864895821.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  train_df.iloc[0][1]


'feels more like a rejected x-files episode than a credible account of a puzzling real-life happening .'

In [8]:
test_df = pd.read_csv("mr/test-sst2.csv", encoding='unicode_escape', header=0)

In [9]:
test_df.shape

(961, 2)

In [10]:
test_df.head()

,sentiment,review
0,1,although german cooking does not come readily ...
1,1,"a very well-made , funny and entertaining pict..."
2,0,coughs and sputters on its own postmodern conc...
3,0,the movie is dawn of the dead crossed with joh...
4,0,this is the sort of burly action flick where o...


In [11]:
train_df['sentiment'] = (train_df['sentiment']).astype(int)
test_df['sentiment'] = (test_df['sentiment']).astype(int)

train_df.head(30)

,sentiment,review
0,0,feels more like a rejected x-files episode tha...
1,0,with spy kids 2 : the island of lost dreams wr...
2,1,"while the story 's undeniably hard to follow ,..."
3,1,"not only is undercover brother as funny , if n..."
4,0,dignified ceo 's meet at a rustic retreat and ...
5,0,even if it made its original release date last...
6,1,a fascinating glimpse into an insular world th...
7,1,far more imaginative and ambitious than the tr...
8,0,-lrb- a -rrb- poorly executed comedy .
9,1,griffiths proves she 's that rare luminary who...


In [12]:
test_df.head()

,sentiment,review
0,1,although german cooking does not come readily ...
1,1,"a very well-made , funny and entertaining pict..."
2,0,coughs and sputters on its own postmodern conc...
3,0,the movie is dawn of the dead crossed with joh...
4,0,this is the sort of burly action flick where o...


In [13]:
train_df_bert = pd.DataFrame({
    "id": range(len(train_df)),
    "label": train_df['sentiment'],
    "sentence": train_df['review'].replace(r'\n', ' ', regex=True)
})

train_df_bert.head()

,id,label,sentence
0,0,0,feels more like a rejected x-files episode tha...
1,1,0,with spy kids 2 : the island of lost dreams wr...
2,2,1,"while the story 's undeniably hard to follow ,..."
3,3,1,"not only is undercover brother as funny , if n..."
4,4,0,dignified ceo 's meet at a rustic retreat and ...


In [14]:
train_df_bert

,id,label,sentence
0,0,0,feels more like a rejected x-files episode tha...
1,1,0,with spy kids 2 : the island of lost dreams wr...
2,2,1,"while the story 's undeniably hard to follow ,..."
3,3,1,"not only is undercover brother as funny , if n..."
4,4,0,dignified ceo 's meet at a rustic retreat and ...
...,...,...,...
8647,8647,1,a superbly acted and funny\/gritty fable of th...
8648,8648,0,we miss the quirky amazement that used to come...
8649,8649,0,a benign but forgettable sci-fi diversion .
8650,8650,1,elvira fans could hardly ask for more .


In [15]:
train_df_bert = train_df_bert.iloc[0:10000]

In [16]:
train_df_bert

,id,label,sentence
0,0,0,feels more like a rejected x-files episode tha...
1,1,0,with spy kids 2 : the island of lost dreams wr...
2,2,1,"while the story 's undeniably hard to follow ,..."
3,3,1,"not only is undercover brother as funny , if n..."
4,4,0,dignified ceo 's meet at a rustic retreat and ...
...,...,...,...
8647,8647,1,a superbly acted and funny\/gritty fable of th...
8648,8648,0,we miss the quirky amazement that used to come...
8649,8649,0,a benign but forgettable sci-fi diversion .
8650,8650,1,elvira fans could hardly ask for more .


In [17]:
test_df_bert = pd.DataFrame({
    "id": range(len(test_df)),
    "label": test_df['sentiment'],
    "sentence": test_df['review'].replace(r'\n', ' ', regex=True)
})

test_df_bert.head()

,id,label,sentence
0,0,1,although german cooking does not come readily ...
1,1,1,"a very well-made , funny and entertaining pict..."
2,2,0,coughs and sputters on its own postmodern conc...
3,3,0,the movie is dawn of the dead crossed with joh...
4,4,0,this is the sort of burly action flick where o...


In [18]:
test_df_bert = test_df_bert.iloc[0:3000]

In [19]:
test_df_bert

,id,label,sentence
0,0,1,although german cooking does not come readily ...
1,1,1,"a very well-made , funny and entertaining pict..."
2,2,0,coughs and sputters on its own postmodern conc...
3,3,0,the movie is dawn of the dead crossed with joh...
4,4,0,this is the sort of burly action flick where o...
...,...,...,...
956,956,0,"its mysteries are transparently obvious , and ..."
957,957,1,a sensitive and expertly acted crowd-pleaser t...
958,958,0,"... too slow , too boring , and occasionally a..."
959,959,0,this is pretty dicey material .


In [20]:
# Get the lists of sentences and their labels.
train_sentences = train_df_bert.sentence.values
train_labels = train_df_bert.label.values

In [21]:
# Get the lists of sentences and their labels.
test_sentences = test_df_bert.sentence.values
test_labels = test_df_bert.label.values

In [22]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [23]:
# Print the original sentence.
print(' Original: ', train_sentences[1])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(train_sentences[1]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_sentences[1])))

 Original:  with spy kids 2 : the island of lost dreams writer\/director\/producer robert rodriguez has cobbled together a film that feels like a sugar high gone awry .
Tokenized:  ['with', 'spy', 'kids', '2', ':', 'the', 'island', 'of', 'lost', 'dreams', 'writer', '\\', '/', 'director', '\\', '/', 'producer', 'robert', 'rodriguez', 'has', 'cobb', '##led', 'together', 'a', 'film', 'that', 'feels', 'like', 'a', 'sugar', 'high', 'gone', 'aw', '##ry', '.']
Token IDs:  [2007, 8645, 4268, 1016, 1024, 1996, 2479, 1997, 2439, 5544, 3213, 1032, 1013, 2472, 1032, 1013, 3135, 2728, 9172, 2038, 17176, 3709, 2362, 1037, 2143, 2008, 5683, 2066, 1037, 5699, 2152, 2908, 22091, 2854, 1012]


In [24]:
import warnings
warnings.filterwarnings('ignore')

max_len_train = 0

# For every sentence...
for sent in train_sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent[:512], add_special_tokens=True)

    # Update the maximum sentence length.
    max_len_train = max(max_len_train, len(input_ids))

print('Max sentence length: ', max_len_train)

Max sentence length:  78


In [25]:
max_len_test = 0

# For every sentence...
for sent in test_sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent[:512], add_special_tokens=True)

    # Update the maximum sentence length.
    max_len_test = max(max_len_test, len(input_ids))

print('Max sentence length: ', max_len_test)

Max sentence length:  63


In [26]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in train_sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 60,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation=True
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(train_labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', train_sentences[0])
print('Token IDs:', input_ids[0])

Original:  feels more like a rejected x-files episode than a credible account of a puzzling real-life happening .
Token IDs: tensor([  101,  5683,  2062,  2066,  1037,  5837,  1060,  1011,  6764,  2792,
         2084,  1037, 23411,  4070,  1997,  1037, 16405, 20838,  2613,  1011,
         2166,  6230,  1012,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])


In [27]:

train_df_bert.head()

,id,label,sentence
0,0,0,feels more like a rejected x-files episode tha...
1,1,0,with spy kids 2 : the island of lost dreams wr...
2,2,1,"while the story 's undeniably hard to follow ,..."
3,3,1,"not only is undercover brother as funny , if n..."
4,4,0,dignified ceo 's meet at a rustic retreat and ...


In [28]:
(x_train, y_train), (x_test, y_test) = (train_df_bert.sentence,train_df_bert.label),(test_df_bert.sentence,test_df_bert.label)

In [29]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [30]:
from keras_preprocessing.sequence import pad_sequences
# 2 - Define the number of words to be used from a review (truncate and pad input sequences)
print('Pad sequences (samples x time)')

max_review_length = 60

x_train = pad_sequences(x_train, maxlen=max_review_length)
print('X_train shape:',x_train.shape)
x_test = pad_sequences(x_test, maxlen=max_review_length)
print('X_test shape:',x_test.shape)

Pad sequences (samples x time)
X_train shape: (8652, 60)
X_test shape: (961, 60)


In [31]:
from keras import backend as K
from tensorflow import keras
import tensorflow as tf

class FuzzyLayer(keras.layers.Layer):

    def __init__(self, 
                 output_dim, 
                 initial_centers=None,
                 initial_sigmas=None, 
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        self.output_dim = output_dim
        self.initial_centers = initial_centers
        self.initial_sigmas = initial_sigmas
        super(FuzzyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_dimensions = list(input_shape)[:-1:-1]

        c_init_values =[]
        
        if self.initial_centers is None:
            c_init_values= tf.random_uniform_initializer(-1, 1)(shape=(input_shape[-1], self.output_dim), dtype="float32")    
        else:
            c_init_values =tf.convert_to_tensor(self.initial_centers, dtype="float32")
        self.c = tf.Variable(initial_value= c_init_values, trainable=True)

        a_init_values = []
        if self.initial_sigmas is None:
            a_init_values = tf.ones_initializer()(shape=(input_shape[-1], self.output_dim), dtype="float32")    
        else:
            a_init_values = tf.convert_to_tensor(self.initial_sigmas, dtype="float32")
        self.a = tf.Variable(initial_value= a_init_values, trainable=True)
        super(FuzzyLayer, self).build(input_shape)  

    def call(self, x):
        
        aligned_x = K.repeat_elements(K.expand_dims(x, axis = -1), self.output_dim, -1)
        aligned_c = self.c
        aligned_a = self.a
        for dim in self.input_dimensions:
            aligned_c = K.repeat_elements(K.expand_dims(aligned_c, 0), dim, 0)
            aligned_a = K.repeat_elements(K.expand_dims(aligned_a, 0), dim, 0)

        xc = K.exp(-K.sum(K.square((aligned_x - aligned_c) / (2 * aligned_a)), axis=-2, keepdims=False))
        
        return xc
        
    def compute_output_shape(self, input_shape):
        return tuple(input_shape[:-1]) + (self.output_dim,)


In [32]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K

class DefuzzyLayer(keras.layers.Layer):

    def __init__(self, 
                 output_dim, 
                 initial_rules_outcomes=None,
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        self.output_dim = output_dim
        self.initial_rules_outcomes = initial_rules_outcomes
        super(DefuzzyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_dimensions = list(input_shape)[:-1:-1]

        outcomes_init_values = []
        if self.initial_rules_outcomes is None:
            outcomes_init_values = tf.random_uniform_initializer()(shape=(input_shape[-1], self.output_dim), dtype="float32")    
        else:
            outcomes_init_values = tf.convert_to_tensor(self.initial_rules_outcomes, dtype="float32")
        
        self.rules_outcome = tf.Variable(initial_value = outcomes_init_values, trainable=True)
        
        super(DefuzzyLayer, self).build(input_shape)  

    def call(self, x):
        aligned_x = K.repeat_elements(K.expand_dims(x, axis = -1), self.output_dim, -1)
        aligned_rules_outcome = self.rules_outcome
        for dim in self.input_dimensions:
            aligned_rules_outcome = K.repeat_elements(K.expand_dims(aligned_rules_outcome, 0), dim, 0)
        
        xc = K.sum((aligned_x * aligned_rules_outcome), axis=-2, keepdims=False)
        return xc
        
    def compute_output_shape(self, input_shape):
        return tuple(input_shape[:-1]) + (self.output_dim,)
    
    def get_config(self):
        return {"rules_outcome": self.rules_outcome.numpy()}


In [33]:

import tensorflow
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation, Flatten
from keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from keras.layers import Conv1D
from keras.layers import GlobalMaxPooling1D,MaxPooling1D
from keras.layers import Embedding
import matplotlib.pyplot as plt
import torch.optim as optim
#from FuzzyLayer import FuzzyLayer
#from DefuzzyLayer import DefuzzyLayer

# 3 - Neural network model design
print('Building model')
max_features = 2000
embedding_vector_length = 32
hidden_dims = 40

model = Sequential()
model.add(Embedding(max_features, embedding_vector_length, input_length=max_review_length))

model.add(Bidirectional(LSTM(30,activation='tanh',
                             recurrent_activation='sigmoid',
                             return_state=False,
                             return_sequences=False)))
model.add(Dropout(0.1))
model.add(FuzzyLayer(20, input_dim=2))
model.add(Dropout(0.1))
model.add(DefuzzyLayer(10))
model.add(Dropout(0.1))
#model.add(Dense(2, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

model.summary()


Building model
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 32)            64000     
_________________________________________________________________
bidirectional (Bidirectional (None, 60)                15120     
_________________________________________________________________
dropout (Dropout)            (None, 60)                0         
_________________________________________________________________
fuzzy_layer (FuzzyLayer)     (None, 20)                2400      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
defuzzy_layer (DefuzzyLayer) (None, 10)                200       
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)  

In [34]:
import numpy
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# 4 - Train the neural network model
history = model.fit(x_train, y_train,
                    epochs=20,
                    verbose=True,
                    callbacks=[es_callback],
                    #validation_data=(x_test, y_test),
                    validation_split=0.1,
                    batch_size=10)

loss, accuracy = model.evaluate(x_train, y_train, verbose=True)
print("Training Accuracy: {:.6f}".format(accuracy))
print("Training Loss: {:.6f}".format(loss))
loss, accuracy = model.evaluate(x_test, y_test, verbose=True)
print("Testing Accuracy:  {:.6f}".format(accuracy))
print("Testing Loss:  {:.6f}".format(loss))


Epoch 1/20
779/779 [==============================] - 19s 17ms/step - loss: 0.6800 - accuracy: 0.5378 - val_loss: 0.6579 - val_accuracy: 0.6455
Epoch 2/20
779/779 [==============================] - 12s 16ms/step - loss: 0.5263 - accuracy: 0.7695 - val_loss: 0.5155 - val_accuracy: 0.7506
Epoch 3/20
779/779 [==============================] - 12s 15ms/step - loss: 0.4073 - accuracy: 0.8270 - val_loss: 0.5169 - val_accuracy: 0.7667
Epoch 4/20
779/779 [==============================] - 12s 15ms/step - loss: 0.3496 - accuracy: 0.8592 - val_loss: 0.5107 - val_accuracy: 0.7691
Epoch 5/20
779/779 [==============================] - 12s 15ms/step - loss: 0.3098 - accuracy: 0.8777 - val_loss: 0.5252 - val_accuracy: 0.7806
Epoch 6/20
779/779 [==============================] - 11s 15ms/step - loss: 0.2701 - accuracy: 0.8910 - val_loss: 0.5361 - val_accuracy: 0.7887
Epoch 7/20
271/271 [==============================] - 8s 29ms/step - loss: 0.2248 - accuracy: 0.9187
Training Accuracy: 0.918747
Trainin